# Set up

In [1]:
!git clone https://github.com/aliswh/lastfm
!cd lastfm; git pull
!cd lastfm; pip install -r requirements.txt

Cloning into 'lastfm'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 171 (delta 84), reused 128 (delta 49), pack-reused 0
Receiving objects: 100% (171/171), 26.36 KiB | 550.00 KiB/s, done.
Resolving deltas: 100% (84/84), done.
Already up to date.
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 198 kB 46.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=699f366156e37aa8b2632754da905c7d5bcf2b4e61a5b32338021797063ab7aa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from py4j.java_gateway import UserHelpAutoCompletion
import pyspark
from pyspark.sql import  SparkSession
from lastfm.src.ingestion_layer.googlestorage import *
from lastfm.src.ingestion_layer.pylastsource import *
from lastfm.src.ingestion_layer.config import *
import json
import datetime

# PySpark Context

In [3]:
sc = pyspark.SparkContext("local[*]")
spark = SparkSession(sc)

storage = GoogleStorage('./lastfm/src/ingestion_layer/creds.json',bucket_name)
source = PyLastSource(AUTH_DATA)

Batch job

In [17]:
SEED_USER = 'dars4'
users = source.get('user_pool',SEED_USER,MAX=3)
for dict_user in users:
    print(dict_user)
    storage.write(json.dumps(dict_user), f"users/user_{dict_user['id']}")
    json_recent_tracks = json.dumps(source.get('recent_tracks',dict_user['user'],limit=10))
    storage.write(json_recent_tracks, f"recent_tracks/user_recent_tracks_{dict_user['id']}")
    dict_recent_tracks = json.loads(json_recent_tracks)
    for track in dict_recent_tracks[1:]:
        print(track)
        json_track = source.get('track', track['artist'], track['title'])
        storage.write(json.dumps(json_track), f"tracks/track_{json_track['id']}")

{'user': 'c4zt96', 'playcount': 11580, 'reg_date': '1627094549', 'country': 'Brazil', 'url': 'https://www.last.fm/user/c4zt96', 'id': '0d84bc5dedacad885f9fa1df8a0b8d8ca8467c80'}
{'artist': 'Queens of the Stone Age', 'title': 'I Appear Missing', 'date': '17 Feb 2022, 05:02', 'album': '...Like Clockwork', 'timestamp': '1645074178', 'id': '8581023ebcd2cef5c8bdfbc6ab008f0bbe7d62bc'}
{'artist': 'Queens of the Stone Age', 'title': 'Smooth Sailing', 'date': '17 Feb 2022, 04:58', 'album': '...Like Clockwork', 'timestamp': '1645073888', 'id': '9aa23307911ed56dd938fc10a370f7ca3c7b8ea4'}
{'artist': 'Queens of the Stone Age', 'title': 'Fairweather Friends', 'date': '17 Feb 2022, 04:54', 'album': '...Like Clockwork', 'timestamp': '1645073664', 'id': '16184acb9c2a4f99becdec220462c72bb5bbfaec'}
{'artist': 'Queens of the Stone Age', 'title': 'Kalopsia', 'date': '17 Feb 2022, 04:49', 'album': '...Like Clockwork', 'timestamp': '1645073386', 'id': '6db3aaf0029b1d8f3881da2f5d653ec5cfe2dc9f'}
{'artist': 'Q

Missing track

In [18]:
source.get('track', 'Rich Nanni', 'Macaroni')

{'album': None,
 'artist': 'Rich Nanni',
 'duration': None,
 'id': 'cb5617ef265b3fe5e48c859d7016a09dcf4a4215',
 'tags': None,
 'title': 'Macaroni',
 'url': 'https://www.last.fm/music/rich%2bnanni/_/macaroni'}

# Read from storage

In [19]:
recent_tracks = storage.read('recent_tracks',dir=True)
for recent_track in recent_tracks:
  print(recent_track)
recent_tracks_rdd = sc.parallelize(recent_tracks)

['amarxl', {'artist': 'Kim Petras', 'title': 'Superpower Bitch', 'date': '17 Feb 2022, 00:17', 'album': 'Slut Pop', 'timestamp': '1645057052', 'id': 'afc99b7f3a550faf05aa7d29efa46bb01d29c4eb'}, {'artist': 'Kim Petras', 'title': 'XXX', 'date': '17 Feb 2022, 00:15', 'album': 'Slut Pop', 'timestamp': '1645056927', 'id': '4918d56108433a5a44194077398663d691a68741'}, {'artist': 'Kim Petras', 'title': 'Treat Me Like a Slut', 'date': '17 Feb 2022, 00:13', 'album': 'Slut Pop', 'timestamp': '1645056810', 'id': '9e41f80918c4172b2b70bbe2bbc3b1042af80290'}, {'artist': 'florentino', 'title': 'Fuego (Florentino Remix)', 'date': '16 Feb 2022, 21:06', 'album': 'Fuego Remixes', 'timestamp': '1645045590', 'id': '543c59f310eaa62d9ca7babbad7f3bed690e2be4'}, {'artist': 'Isabella Lovestory', 'title': 'Mariposa', 'date': '16 Feb 2022, 21:03', 'album': 'Mariposa', 'timestamp': '1645045410', 'id': '8bca9f3eb3940485e934f71a9d118f00960dc849'}, {'artist': 'ROSALÍA', 'title': 'SAOKO', 'date': '16 Feb 2022, 21:01', 

In [90]:
tracks = storage.read('tracks',dir=True)
tracks_rdd = sc.parallelize(tracks)
tracks_rdd.take(10)

[{'album': 'Pretty Boy, Dirty Boy',
  'artist': 'Maluma',
  'duration': 206000,
  'id': '0020e4478d241191c22f7d9836167ca27add0ea1',
  'tags': {'2010s': '505fbc527f32c051a6d02745d282bdc333b7cf5d',
   '2015': '9cdda67ded3f25811728276cefa76b80913b4c54',
   'Reggaeton': '1e32500a2ea9d27eccb199f4efb8d581a9216aad'},
  'title': 'El Perdedor',
  'url': 'https://www.last.fm/music/maluma/_/el%2bperdedor'},
 {'album': 'Life Support',
  'artist': 'Madison Beer',
  'duration': 164000,
  'id': '04eb3a46eef58880440618cd00e1c55034020e38',
  'tags': {'alternative': '816d63e63c53fbbcdc73b605731fb850111597ab',
   'dark pop': '35bc49ec028b7658e999e71c36c0276ff2360e24',
   'hipnotizing': '1a2c6275cf375cb706d644577d1ac46135df2ec6',
   'pop': '4f197c99a78b8411f1cf48ab409a0a6d176b99b7',
   'pop alternative': '8db6fef9fef3061b4e84c4b6433656640fad5c9f'},
  'title': 'Stay Numb and Carry On',
  'url': 'https://www.last.fm/music/madison%2bbeer/_/stay%2bnumb%2band%2bcarry%2bon'},
 {'album': 'Ancient Dreams In A Mod

# User features

1. Extract listening sessions from users:
  * list of concatenated songs with a play events +-10 sec after the actual song completed
2. Know statistics about these sessions:
  * average number of tracks
  * average session per user
  * session lengths

In [264]:
def to_datetime(x):
  to_date = lambda ts: datetime.datetime.strptime(ts, '%d %b %Y, %H:%M')
  x[1]['date'] = to_date(x[1]['date'])
  return x

sessions = recent_tracks_rdd.flatMap(lambda x: [(x[0],y) for y in x[1:]]) \
                            .map(lambda x: to_datetime(x))
sessions.take(1)  

[('amarxl',
  {'album': 'Slut Pop',
   'artist': 'Kim Petras',
   'date': datetime.datetime(2022, 2, 17, 0, 17),
   'id': 'afc99b7f3a550faf05aa7d29efa46bb01d29c4eb',
   'timestamp': '1645057052',
   'title': 'Superpower Bitch'})]

In [265]:
def timestamp_to_delta(ts1, ts2):
  return int((ts2 - ts1).total_seconds() * 1000)
  
sessions = sessions.join(sessions) \
                   .filter(lambda x: x[1][0]['date'] < x[1][1]['date']) \
                   .map(lambda x: ((x[0], x[1][0]['title'], x[1][0]['artist'], x[1][1]['title'], x[1][1]['artist'], x[1][0]['date'], x[1][1]['date']), timestamp_to_delta(x[1][0]['date'], x[1][1]['date']))) \
                   .reduceByKey(lambda x,y: min(x,y))

sessions.take(1)  

[(('bellagecs',
   'this is how i learn to say no',
   'Emeline',
   'what it means to be a girl',
   'Emeline',
   datetime.datetime(2022, 2, 17, 4, 13),
   datetime.datetime(2022, 2, 17, 4, 17)),
  240000)]

In [266]:
sessions = sessions.map(lambda x: ((x[0][1], x[0][2]), (x[0][0], x[1])))
sessions.take(1)

[(('this is how i learn to say no', 'Emeline'), ('bellagecs', 240000))]

In [267]:
tr_rdd = tracks_rdd.map(lambda x: ((x['title'],x['artist']),x['duration']))
tr_rdd.take(1)

[(('El Perdedor', 'Maluma'), 206000)]

In [272]:
sessions_tracks = sessions.join(tr_rdd)\
                          .mapValues(lambda x: (x[0][0], x[0][1], x[1]))
sessions_tracks.take(10) # sx from listening session, dx from track

[(('El Consejo', 'Ms Nina'), ('amarxl', 12600000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 12360000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 420000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 960000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 120000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 12480000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 1140000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 840000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 600000, 129000)),
 (('XXX', 'Kim Petras'), ('bellagecs', 360000, 0))]

In [273]:
sessions_tracks = sessions_tracks.filter(lambda x: x[1][2] != 0)
sessions_tracks.take(10)

[(('El Consejo', 'Ms Nina'), ('amarxl', 12600000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 12360000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 420000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 960000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 120000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 12480000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 1140000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 840000, 129000)),
 (('El Consejo', 'Ms Nina'), ('amarxl', 600000, 129000)),
 (('My God Is the Sun', 'Queens of the Stone Age'),
  ('c4zt96', 780000, 235000))]

In [274]:
sessions_tracks = sessions_tracks.map(lambda x: ((x[0], x[1][0]), (x[1][2] - x[1][1] )//1000 + 10 )).filter(lambda x: x[1] >= 0)
sessions_tracks.take(100)

[((('El Consejo', 'Ms Nina'), 'amarxl'), 19),
 ((('My God Is the Sun', 'Queens of the Stone Age'), 'c4zt96'), 5),
 ((('The Vampyre of Time and Memory', 'Queens of the Stone Age'), 'c4zt96'),
  44),
 ((('Mariposa', 'Isabella Lovestory'), 'amarxl'), 9),
 ((('CELEBRÉ', 'Nathy Peluso'), 'amarxl'), 46),
 ((('justified', 'Kacey Musgraves'), 'bellagecs'), 70),
 ((('Smooth Sailing', 'Queens of the Stone Age'), 'c4zt96'), 61),
 ((('If I Had a Tail', 'Queens of the Stone Age'), 'c4zt96'), 5),
 ((('I Sat by the Ocean', 'Queens of the Stone Age'), 'c4zt96'), 5),
 ((('Keep Your Eyes Peeled', 'Queens of the Stone Age'), 'c4zt96'), 14),
 ((('what it means to be a girl', 'Emeline'), 'bellagecs'), 58)]